# Predictive KPI 02  predicted price

### Import libraries

In [40]:
from utils import *

from time import strftime, gmtime
from datetime import datetime
import os
from hdfs import InsecureClient

import re
import os
from time import strftime, gmtime
from datetime import datetime
import shutil

import findspark
import warnings
from pyspark.sql.functions import split, col, avg, count
from pyspark.ml.feature import FeatureHasher

import pyspark as py
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

### Read login information

In [41]:
%run utils.py

log = log_config("My.log")
logging_info = logging_creation()

user_name = logging_info["user_name"]
host = logging_info["host"]

### Download data from formatted zone

In [42]:
%run data_SelectorsAndFormatters.py

inp_path="model_temp"
master_files = {
    "hotels": {"Description": "Information about hotels in neighbourhoods"},
    "renda_familiar": {
        "keywords": ["renda_familiar"],
        "Description": "Data from Open Barcelona with incomes of families. Can be joined with 'idealista' file using a lookup file",
    },
    "idealista": {
        "keywords": ["idealista"],
        "Description": "Appartments from idealista. Can be joined with 'renta familiar' file using a lookup file",
    },
    "lookup_renta_idealista": {
        "keywords": ["extended"],
        "Description": "Lookup datable to join 'Idealista' and 'renda familiar'",
    },
}

In [43]:
# Load last versions of all dataframes from explotation zone
for key_file in master_files.keys():
    formatted_data_selector(log, user_name, host, key_file)

### Load dataframes

In [44]:
from pyspark.sql import SparkSession

findspark.init("/Users/yyf/Documents/Spark")

conf = (
    SparkConf()
        .set("spark.master", "local")
        .set("spark.app.name", "Formatted zone loader")
    )
spark = SparkSession.builder.config(conf=conf).getOrCreate()
log.info("Spark sesion correctly initialized")

# Load all dataframes in a dictionary
dfs = {}
file_names = os.listdir(inp_path)

for file in file_names:
    full_in_path = inp_path + "/" + file

    # Load dataframes
    short_name = file.split("-", maxsplit=2)[2]
    short_name = "".join(short_name)

    dfs[short_name] = (
            spark.read.option("multiline", "true").format("parquet").load(full_in_path)
    )


In [63]:
idealista_path = "model_temp/20240614-2306-idealista"
df_idealista = spark.read.parquet(idealista_path)
df_idealista.show(5)

+------------+--------------------+-----------------+---------+-----+--------+------------+---------+-----+--------+-----+---------+--------------------+---------+--------------------+------------------+-------+----------+---------+-----------+--------------------+--------+--------+------+--------------+-------+-----------+--------------------+--------------------+-------+---------+------+----------+-----------------+----------------------+------------------+------------+---------------------------------------------------------------------------------------------------------------------------+
|propertyCode|           thumbnail|externalReference|numPhotos|floor|   price|propertyType|operation| size|exterior|rooms|bathrooms|             address| province|        municipality|          district|country|  latitude|longitude|showAddress|                 url|distance|hasVideo|status|newDevelopment|hasLift|priceByArea|        detailedType|      suggestedTexts|hasPlan|has3DTour|has360|hasStag

### to explore the data

In [47]:
column_names = df_idealista.columns
print(column_names)

['propertyCode', 'thumbnail', 'externalReference', 'numPhotos', 'floor', 'price', 'propertyType', 'operation', 'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province', 'municipality', 'district', 'country', 'latitude', 'longitude', 'showAddress', 'url', 'distance', 'hasVideo', 'status', 'newDevelopment', 'hasLift', 'priceByArea', 'detailedType', 'suggestedTexts', 'hasPlan', 'has3DTour', 'has360', 'hasStaging', 'topNewDevelopment', 'newDevelopmentFinished', 'parkingSpace', 'neighborhood', 'district,neighborhood,district_n_reconciled,district_n,district_id,neighborhood_n_reconciled,neighborhood_n,neighborhood_id']


In [48]:
unique_operations = df_idealista.select("operation").distinct()
unique_operations.show()

+---------+
|operation|
+---------+
|     sale|
|     NULL|
+---------+



In [49]:
unique_operations = df_idealista.select("propertyType").distinct()
unique_operations.show()

+------------+
|propertyType|
+------------+
|   penthouse|
|      duplex|
|      studio|
|      chalet|
|        flat|
|countryHouse|
|        NULL|
+------------+



In [28]:
unique_operations = df_idealista.select("exterior").distinct()
unique_operations.show()

+--------+
|exterior|
+--------+
|    true|
|   false|
|    NULL|
+--------+



In [29]:
unique_operations = df_idealista.select("district").distinct()
unique_operations.show()

+--------------------+
|            district|
+--------------------+
|La Miranda - Can ...|
|           Collblanc|
|           Can Matas|
|                Golf|
|              Gràcia|
|            El Pedró|
|La Florida - Les ...|
|             El Gall|
|          Sant Martí|
| Roses - Castellbell|
|    Parc Empresarial|
|         Can Vidalet|
|       Can Sant Joan|
|Camps Blancs - Ca...|
|           Les Corts|
|         Granvia L-H|
|              Almeda|
|Sant Francesc-El ...|
|       Sant Ildefons|
|Mas Lluí - Els Mi...|
+--------------------+
only showing top 20 rows



In [30]:
unique_operations = df_idealista.select("status").distinct()
unique_operations.show()

+--------------+
|        status|
+--------------+
|         renew|
|newdevelopment|
|          good|
|          NULL|
+--------------+



In [31]:
unique_operations = df_idealista.select("bathrooms").distinct()
unique_operations.show()

+---------+
|bathrooms|
+---------+
|       19|
|        0|
|        7|
|        6|
|        9|
|        5|
|        1|
|       10|
|        3|
|        8|
|       11|
|        2|
|        4|
|     NULL|
+---------+



## Preprocessing

In [50]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder

# Drop duplicates
df_idealista = df_idealista.dropDuplicates()

# Prepare the DataFrame
df_idealista = df_idealista.dropna(subset=['price', 'size', 'rooms', 'bathrooms', 'status', 'exterior'])
df_idealista = df_idealista.withColumn("price", col("price").cast("double")) \
                           .withColumn("size", col("size").cast("double")) \
                           .withColumn("rooms", col("rooms").cast("integer")) \
                           .withColumn("bathrooms", col("bathrooms").cast("integer")) \
                           .withColumn("exterior", col("exterior").cast("integer"))   # exterior: true is mapped to 1 and false is mapped to 0

# StringIndexer and OneHotEncoder for 'status'
# Ensure no column name conflicts
if "status_indexed" in df_idealista.columns:
    df_idealista = df_idealista.drop("status_indexed")
status_indexer = StringIndexer(inputCol="status", outputCol="status_indexed")
df_idealista = status_indexer.fit(df_idealista).transform(df_idealista)

if "status_encoded" in df_idealista.columns:
    df_idealista = df_idealista.drop("status_encoded")
status_encoder = OneHotEncoder(inputCols=["status_indexed"], outputCols=["status_encoded"])
df_idealista = status_encoder.fit(df_idealista).transform(df_idealista)



if "features" in df_idealista.columns:
    df_idealista = df_idealista.drop("features")
assembler = VectorAssembler(inputCols=["size", "rooms", "bathrooms", "status_encoded", "exterior"], outputCol="features")
df_idealista = assembler.transform(df_idealista)


In [59]:
# Select the specific columns to create a new DataFrame
dfs2 = df_idealista.select("price", "size", "rooms", "bathrooms", "status", "exterior")

### Upload preprocessed dataframe to the explotation zone

In [60]:
model_name = "LM_V2.parquet"
explotation_zone_path = "user/bdm/Model_explotation_zone"

In [61]:
hdfs_client = InsecureClient(host, user=user_name)
out_full_path = explotation_zone_path + "/" + model_name 

dfs2.write.mode('overwrite').parquet(model_name)
hdfs_client.upload(out_full_path, model_name, overwrite=True)

log.info(f"Model {file} uploaded correctly at '{out_full_path}' path")

# Model

## the final model

without scaling the size 

In [58]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Linear Regression Model
lr = LinearRegression(featuresCol="features", labelCol="price")

# Data split for training and testing
train_data, test_data = df_idealista.randomSplit([0.8, 0.2], seed=1234)

# Model training
lr_model = lr.fit(train_data)

# Making predictions
predictions = lr_model.transform(test_data)

# Model evaluation
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

# Displaying coefficients
coefficients = lr_model.coefficients
print("Coefficients:", coefficients)

Root Mean Squared Error (RMSE) on test data = 381345
Coefficients: [0.5304752989772031,40571.224435326534,390222.5468879523,149407.88715313052,187116.91742226458,-8371.780164923639]


In [64]:
# # Scaling
# # We used the same method below with scaling. but chose to unscale all variables for ease of interpretation.
# from pyspark.sql.functions import col
# from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, StandardScaler
# from pyspark.ml.regression import LinearRegression
# from pyspark.ml.evaluation import RegressionEvaluator
# 
# # Drop duplicates based on all columns (if rows are completely identical)
# df_idealista = df_idealista.dropDuplicates()
# 
# # Prepare the DataFrame
# df_idealista = df_idealista.dropna(subset=['price', 'size', 'rooms', 'bathrooms', 'status', 'exterior'])
# df_idealista = df_idealista.withColumn("price", col("price").cast("double")) \
#                            .withColumn("size", col("size").cast("double")) \
#                            .withColumn("rooms", col("rooms").cast("integer")) \
#                            .withColumn("bathrooms", col("bathrooms").cast("integer")) \
#                            .withColumn("exterior", col("exterior").cast("integer"))
# 
# 
# # StringIndexer and OneHotEncoder for 'status'
# if "status_indexed" in df_idealista.columns:
#     df_idealista = df_idealista.drop("status_indexed")
# status_indexer = StringIndexer(inputCol="status", outputCol="status_indexed")
# df_idealista = status_indexer.fit(df_idealista).transform(df_idealista)
# 
# if "status_encoded" in df_idealista.columns:
#     df_idealista = df_idealista.drop("status_encoded")
# status_encoder = OneHotEncoder(inputCols=["status_indexed"], outputCols=["status_encoded"])
# df_idealista = status_encoder.fit(df_idealista).transform(df_idealista)
# 
# # Scale only the 'size' feature
# if "size_features" in df_idealista.columns:
#     df_idealista = df_idealista.drop("size_features")
# size_assembler = VectorAssembler(inputCols=["size"], outputCol="size_features")
# size_df = size_assembler.transform(df_idealista)
# 
# if "scaled_size" in df_idealista.columns:
#     df_idealista = df_idealista.drop("scaled_size")
# scaler = StandardScaler(inputCol="size_features", outputCol="scaled_size", withStd=True, withMean=False)
# df_idealista = scaler.fit(size_df).transform(size_df)
# 
# # Assemble other features
# if "other_features" in df_idealista.columns:
#     df_idealista = df_idealista.drop("other_features")
# other_features_assembler = VectorAssembler(inputCols=["rooms", "bathrooms", "status_encoded", "exterior"], outputCol="other_features")
# df_idealista = other_features_assembler.transform(df_idealista)
# 
# # Combine scaled size with other features
# if "features" in df_idealista.columns:
#     df_idealista = df_idealista.drop("features")
# final_assembler = VectorAssembler(inputCols=["scaled_size", "other_features"], outputCol="features")
# df_idealista = final_assembler.transform(df_idealista)
# 
# # Linear Regression Model using combined features
# lr = LinearRegression(featuresCol="features", labelCol="price")
# 
# # Split the data
# train_data, test_data = df_idealista.randomSplit([0.8, 0.2], seed=1234)
# 
# # Train the model
# lr_model = lr.fit(train_data)
# 
# # Make predictions
# predictions = lr_model.transform(test_data)
# 
# # Evaluate the model
# evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
# rmse = evaluator.evaluate(predictions)
# print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
# 
# # Display coefficients
# coefficients = lr_model.coefficients
# print("Coefficients:", coefficients)

Root Mean Squared Error (RMSE) on test data = 402209
Coefficients: [774.4352699404948,37961.09367814587,383370.4673475959,141776.6014140781,178639.25549421436,-8690.246685712456]
